In [1]:
import numpy as np
import os, yaml, sys
ENV = os.getenv("MY_ENV", "dev")
with open("../../config.yaml", "r") as f:
    config = yaml.safe_load(f)
paths = config[ENV]["paths"]
sys.path.append(paths["src_path"])
from alignment.CKA import cka, cka_minibatch, cka_batch_collection
from sklearn.metrics.pairwise import pairwise_kernels
import torch
import torch.nn.functional as F

In [2]:
# Random activations from two layers
X = np.random.randn(3000, 1000)  # 100 samples, 128 neurons
Y = 5*X +2*np.random.randn(3000, 1000)  # 100 samples, 256 neurons

# Linear CKA
print("Linear CKA:", cka(X, Y, kernel="linear"))

# RBF CKA
print("RBF CKA:", cka(X, Y, kernel="rbf", gamma=1.0))

Linear CKA: 0.8965438447003639
RBF CKA: 0.9999955030298343


In [3]:
cka_scores = []
prev_idx = 0
xy, xx, yy = 0, 0, 0
n_pts = X.shape[0]
batch_size = n_pts
batch_num = n_pts // batch_size
idx = np.random.choice(np.arange(n_pts), size=n_pts, replace=False)
prev_idx = 0
counter = 0
for i in range(batch_size, n_pts+batch_size, batch_size):  # iterate over minibatches
    i = min(i, n_pts)
    #X_batch, Y_batch = X_batch.cuda(), Y_batch.cuda()
    #idx = np.random.choice(np.arange(n_pts), size=batch_size, replace=False)
    #X_batch, Y_batch = X[idx, :], Y[idx, :]
    #print("batch start", prev_idx, "batch end", i-1)
    counter +=1
    #print(counter)
    rand_idx = idx[prev_idx:i]
    X_batch, Y_batch = X[rand_idx, :], Y[rand_idx, :]
    prev_idx = i
    xy_n, xx_n, yy_n = cka_minibatch(X_batch, Y_batch, kernel="linear")
    xy += xy_n
    xx += xx_n
    yy += yy_n
#cka_value = np.mean(np.stack(cka_scores))
print("Estimated minibatch CKA:", cka_batch_collection(xy, xx, yy))

Estimated minibatch CKA: 0.8620530945189993
